In [5]:
import os
from dotenv import load_dotenv
from urllib.parse import urlparse
import psycopg2

load_dotenv()
username = os.environ.get("POSTGRESQL_LOCAL_USER")
password = os.environ.get("POSTGRESQL_LOCAL_PASSWORD")
database = os.environ.get("POSTGRESQL_LOCAL_DATABASE")
hostname = "localhost"
port = int(os.environ.get("POSTGRESQL_LOCAL_PORT"))

connection = psycopg2.connect(
    database = database,
    user = username,
    password = password,
    host = hostname,
    port = port
)

In [6]:
import psycopg2
import timeit
from datetime import datetime

def create_username_history_table(conn):
    cur = conn.cursor()

    cur.execute("""
        CREATE TABLE IF NOT EXISTS username_history_from_app (
            username_history_id SERIAL PRIMARY KEY,
            user_id INTEGER,
            username_new VARCHAR(255),
            timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)

    conn.commit()
    cur.close()

def update_with_trigger(conn):
    cur = conn.cursor()

    user_id = 1
    new_username = "nouveau_username"

    cur.execute("UPDATE user_table SET username = %s WHERE user_id = %s", (new_username, user_id))

    conn.commit()
    cur.close()

# Fonction pour mettre à jour le username sans utiliser le trigger
def update_without_trigger(conn):
    cur = conn.cursor()

    user_id = 1
    new_username = "nouveau_username"

    timestamp = datetime.now()
    cur.execute("INSERT INTO username_history_from_app (user_id, username_new, timestamp) VALUES (%s, %s, %s)",
                (user_id, new_username, timestamp))

    conn.commit()
    cur.close()

create_username_history_table(connection)

time_with_trigger_100 = timeit.timeit(lambda: update_with_trigger(connection), number=100)

time_without_trigger_100 = timeit.timeit(lambda: update_without_trigger(connection), number=100)

time_with_trigger_1000 = timeit.timeit(lambda: update_with_trigger(connection), number=1000)

time_without_trigger_1000 = timeit.timeit(lambda: update_without_trigger(connection), number=1000)

time_with_trigger_100000 = timeit.timeit(lambda: update_with_trigger(connection), number=100000)

time_without_trigger_100000 = timeit.timeit(lambda: update_without_trigger(connection), number=100000)

connection.close()
print(f"Temps avec le trigger : {time_with_trigger_100} secondes")
print(f"Temps sans le trigger : {time_without_trigger_100} secondes")
print(f"Temps avec le trigger : {time_with_trigger_1000} secondes")
print(f"Temps sans le trigger : {time_without_trigger_1000} secondes")
print(f"Temps avec le trigger : {time_with_trigger_100000} secondes")
print(f"Temps sans le trigger : {time_without_trigger_100000} secondes")


Temps avec le trigger : 0.419237431982765 secondes
Temps sans le trigger : 0.3172601420083083 secondes
Temps avec le trigger : 3.5829878650256433 secondes
Temps sans le trigger : 3.063297307002358 secondes
Temps avec le trigger : 324.2239217199967 secondes
Temps sans le trigger : 331.4469891930057 secondes
